# CSE488
## Spark Tutorial in pySpark

In this tutorial you will learn how to use [Apache Spark](https://spark.apache.org) in local mode on a Colab enviroment.

Credits to [Tiziano Piccardi](http://piccardi.me/) for his Spark Tutorial used in the Applied Data Analysis class at EPFL.

### Setup

Let's setup Spark on your Colab environment.  Run the cell below!

In [1]:
!apt autoremove

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
0 upgraded, 0 newly installed, 0 to remove and 6 not upgraded.


In [2]:
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.9/316.9 MB 2.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.0-py2.py3-none-any.whl size=317425344 sha256=ef2311d3b6addb4c73f07fb49cbff894750b2ba734ab9ea436d8fc42e76cdd3d
  Stored in directory: /root/.cache/pip/wheels/41/4e/10/c2cf2467f71c678cfc8a6b9ac9241e5e44a01940da8fbb17fc
Successfully built pyspark


In [3]:
!pip install -U -q PyDrive

In [4]:
!apt install openjdk-8-jdk-headless -qq

The following additional packages will be installed:
  libxtst6 openjdk-8-jre-headless
Suggested packages:
  openjdk-8-demo openjdk-8-source libnss-mdns fonts-dejavu-extra fonts-nanum fonts-ipafont-gothic
  fonts-ipafont-mincho fonts-wqy-microhei fonts-wqy-zenhei fonts-indic
The following NEW packages will be installed:
  libxtst6 openjdk-8-jdk-headless openjdk-8-jre-headless
0 upgraded, 3 newly installed, 0 to remove and 6 not upgraded.
Need to get 39.7 MB of archives.
After this operation, 144 MB of additional disk space will be used.
Selecting previously unselected package libxtst6:amd64.
(Reading database ... 120882 files and directories currently installed.)
Preparing to unpack .../libxtst6_2%3a1.2.3-1build4_amd64.deb ...
Unpacking libxtst6:amd64 (2:1.2.3-1build4) ...
Selecting previously unselected package openjdk-8-jre-headless:amd64.
Preparing to unpack .../openjdk-8-jre-headless_8u382-ga-1~22.04.1_amd64.deb ...
Unpacking openjdk-8-jre-headless:amd64 (8u382-ga-1~22.04.1) ...
Se

In [5]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

Now we authenticate a Google Drive client to download the file we will be processing in our Spark job.

**Make sure to follow the interactive instructions.**

In [6]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [7]:
id='1L6pCQkldvdBoaEhRFzL0VnrggEFvqON4'
downloaded = drive.CreateFile({'id': id})
downloaded.GetContentFile('Bombing_Operations.json.gz')

id='14dyBmcTBA32uXPxDbqr0bFDIzGxMTWwl'
downloaded = drive.CreateFile({'id': id})
downloaded.GetContentFile('Aircraft_Glossary.json.gz')

In [8]:
# Let's import the libraries we will need
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

import pyspark
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext, SparkConf

Let's initialize the Spark context.

In [9]:
# create the session
conf = SparkConf().set("spark.ui.port", "4050")

# create the context
sc = pyspark.SparkContext(conf=conf)
spark = SparkSession.builder.getOrCreate()

You can easily check the current version and get the link of the web interface. In the Spark UI, you can monitor the progress of your job and debug the performance bottlenecks (if your Colab is running with a **local runtime**).

In [10]:
spark

If you are running this Colab on the Google hosted runtime, the cell below will create a ngrok tunnel which will allow you to still check the Spark UI.

In [11]:
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip ngrok-stable-linux-amd64.zip
get_ipython().system_raw('./ngrok http 4050 &')
!curl -s http://localhost:4040/api/tunnels | python3 -c \
    "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"

--2023-11-15 07:41:33--  https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
Resolving bin.equinox.io (bin.equinox.io)... 54.237.133.81, 18.205.222.128, 52.202.168.65, ...
Connecting to bin.equinox.io (bin.equinox.io)|54.237.133.81|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13921656 (13M) [application/octet-stream]
Saving to: ‘ngrok-stable-linux-amd64.zip’

ngrok-stable-linux- 100%[===================>]  13.28M  5.54MB/s    in 2.4s    

2023-11-15 07:41:37 (5.54 MB/s) - ‘ngrok-stable-linux-amd64.zip’ saved [13921656/13921656]

Archive:  ngrok-stable-linux-amd64.zip
  inflating: ngrok                   
Traceback (most recent call last):
  File "<string>", line 1, in <module>
  File "/usr/lib/python3.10/json/__init__.py", line 293, in load
    return loads(fp.read(),
  File "/usr/lib/python3.10/json/__init__.py", line 346, in loads
    return _default_decoder.decode(s)
  File "/usr/lib/python3.10/json/decoder.py", line 337, in decode
    obj,

# Vietnam War

**Pres. Johnson**: _What do you think about this Vietnam thing? I’d like to hear you talk a little bit._

**Sen. Russell**: _Well, frankly, Mr. President, it’s the damn worse mess that I ever saw, and I don’t like to brag and I never have been right many times in my life, but I knew that we were going to get into this sort of mess when we went in there._

May 27, 1964

![banner](https://raw.githubusercontent.com/epfl-ada/2019/c17af0d3c73f11cb083717b7408fedd86245dc4d/Tutorials/04%20-%20Scaling%20Up/img/banner.jpg)

----

The Vietnam War, also known as the Second Indochina War, and in Vietnam as the Resistance War Against America or simply the American War, was a conflict that occurred in Vietnam, Laos, and Cambodia from 1 November 1955 to the fall of Saigon on 30 April 1975. It was the second of the Indochina Wars and was officially fought between North Vietnam and the government of South Vietnam.

**The dataset describes all the air force operation in during the Vietnam War.**

**Bombing_Operations** [Get the dataset here](https://drive.google.com/a/epfl.ch/file/d/1L6pCQkldvdBoaEhRFzL0VnrggEFvqON4/view?usp=sharing)

- AirCraft: _Aircraft model (example: EC-47)_
- ContryFlyingMission: _Country_
- MissionDate: _Date of the mission_
- OperationSupported: _Supported War operation_ (example: [Operation Rolling Thunder](https://en.wikipedia.org/wiki/Operation_Rolling_Thunder))
- PeriodOfDay: _Day or night_
- TakeoffLocation: _Take off airport_
- TimeOnTarget
- WeaponType
- WeaponsLoadedWeight

**Aircraft_Glossary** [Get the dataset here](https://drive.google.com/a/epfl.ch/file/d/14dyBmcTBA32uXPxDbqr0bFDIzGxMTWwl/view?usp=sharing)

- AirCraft: _Aircraft model (example: EC-47)_
- AirCraftName
- AirCraftType

**Dataset Information:**

THOR is a painstakingly cultivated database of historic aerial bombings from World War I through Vietnam. THOR has already proven useful in finding unexploded ordnance in Southeast Asia and improving Air Force combat tactics:
https://www.kaggle.com/usaf/vietnam-war-bombing-operations

Loading the datset

In [12]:
Bombing_operation =spark.read.json("Bombing_Operations.json.gz")

In [13]:
Aircraft_glossary = spark.read.json("Aircraft_Glossary.json.gz")

Showing the datset

In [14]:
Bombing_operation.show()

+--------+--------------------+-----------+------------------+-----------+---------------+-------------+------------+--------------------+-------------------+
|AirCraft| ContryFlyingMission|MissionDate|OperationSupported|PeriodOfDay|TakeoffLocation|TargetCountry|TimeOnTarget|          WeaponType|WeaponsLoadedWeight|
+--------+--------------------+-----------+------------------+-----------+---------------+-------------+------------+--------------------+-------------------+
|   EC-47|UNITED STATES OF ...| 1971-06-05|              NULL|          D|   TAN SON NHUT|     CAMBODIA|      1005.0|                NULL|                  0|
|   EC-47|UNITED STATES OF ...| 1972-12-26|              NULL|          D|  NAKHON PHANOM|SOUTH VIETNAM|       530.0|                NULL|                  0|
|    RF-4|UNITED STATES OF ...| 1973-07-28|              NULL|          D|       UDORN AB|         LAOS|       730.0|                NULL|                  0|
|     A-1|UNITED STATES OF ...| 1970-02-02|   

In [15]:
Bombing_operation.take(5)

[Row(AirCraft='EC-47', ContryFlyingMission='UNITED STATES OF AMERICA', MissionDate='1971-06-05', OperationSupported=None, PeriodOfDay='D', TakeoffLocation='TAN SON NHUT', TargetCountry='CAMBODIA', TimeOnTarget=1005.0, WeaponType=None, WeaponsLoadedWeight=0),
 Row(AirCraft='EC-47', ContryFlyingMission='UNITED STATES OF AMERICA', MissionDate='1972-12-26', OperationSupported=None, PeriodOfDay='D', TakeoffLocation='NAKHON PHANOM', TargetCountry='SOUTH VIETNAM', TimeOnTarget=530.0, WeaponType=None, WeaponsLoadedWeight=0),
 Row(AirCraft='RF-4', ContryFlyingMission='UNITED STATES OF AMERICA', MissionDate='1973-07-28', OperationSupported=None, PeriodOfDay='D', TakeoffLocation='UDORN AB', TargetCountry='LAOS', TimeOnTarget=730.0, WeaponType=None, WeaponsLoadedWeight=0),
 Row(AirCraft='A-1', ContryFlyingMission='UNITED STATES OF AMERICA', MissionDate='1970-02-02', OperationSupported=None, PeriodOfDay='N', TakeoffLocation='NAKHON PHANOM', TargetCountry='LAOS', TimeOnTarget=1415.0, WeaponType='BLU

Showing the schema of JSON file

In [16]:
Bombing_operation.printSchema()

root
 |-- AirCraft: string (nullable = true)
 |-- ContryFlyingMission: string (nullable = true)
 |-- MissionDate: string (nullable = true)
 |-- OperationSupported: string (nullable = true)
 |-- PeriodOfDay: string (nullable = true)
 |-- TakeoffLocation: string (nullable = true)
 |-- TargetCountry: string (nullable = true)
 |-- TimeOnTarget: double (nullable = true)
 |-- WeaponType: string (nullable = true)
 |-- WeaponsLoadedWeight: long (nullable = true)



In [17]:
Aircraft_glossary.show(80)

+--------+--------------------+--------------------+
|AirCraft|        AirCraftName|        AirCraftType|
+--------+--------------------+--------------------+
|     A-1|Douglas A-1 Skyra...|         Fighter Jet|
|    A-26|Douglas A-26 Invader|        Light Bomber|
|    A-37|Cessna A-37 Drago...|Light ground-atta...|
|     A-4|McDonnell Douglas...|         Fighter Jet|
|     A-5|North American A-...|          Bomber Jet|
|     A-6|Grumman A-6 Intruder|     Attack Aircraft|
|     A-7|  LTV A-7 Corsair II|     Attack Aircraft|
|  AC-119|Fairchild AC-119 ...|Military Transpor...|
|  AC-123|Fairchild C-123 P...|Military Transpor...|
|  AC-130|Lockheed AC-130 S...|Fixed wing ground...|
|   AC-47|Douglas AC-47 Spooky|Ground attack air...|
|    AH-1| Bell AH-1 HueyCobra|          Helicopter|
|     B-1| Rockwell B-1 Lancer|Heavy strategic b...|
|    B-52| B-52 Stratofortress|    Strategic bomber|
|    B-57|Martin B-57 Canberra|     Tactical Bomber|
|    B-66|Douglas B-66 Dest...|        Light B

In [18]:
Aircraft_glossary.take(5)

[Row(AirCraft='A-1', AirCraftName='Douglas A-1 Skyraider', AirCraftType='Fighter Jet'),
 Row(AirCraft='A-26', AirCraftName='Douglas A-26 Invader', AirCraftType='Light Bomber'),
 Row(AirCraft='A-37', AirCraftName='Cessna A-37 Dragonfly', AirCraftType='Light ground-attack aircraft'),
 Row(AirCraft='A-4', AirCraftName='McDonnell Douglas A-4 Skyhawk', AirCraftType='Fighter Jet'),
 Row(AirCraft='A-5', AirCraftName='North American A-5 Vigilante', AirCraftType='Bomber Jet')]

In [19]:
Aircraft_glossary.printSchema()

root
 |-- AirCraft: string (nullable = true)
 |-- AirCraftName: string (nullable = true)
 |-- AirCraftType: string (nullable = true)



Executing The Query-SELECT

In [20]:
Bombing_operation.select("AirCraft","ContryFlyingMission","MissionDate").show()

+--------+--------------------+-----------+
|AirCraft| ContryFlyingMission|MissionDate|
+--------+--------------------+-----------+
|   EC-47|UNITED STATES OF ...| 1971-06-05|
|   EC-47|UNITED STATES OF ...| 1972-12-26|
|    RF-4|UNITED STATES OF ...| 1973-07-28|
|     A-1|UNITED STATES OF ...| 1970-02-02|
|    A-37|     VIETNAM (SOUTH)| 1970-10-08|
|     F-4|UNITED STATES OF ...| 1970-11-25|
|     A-4|UNITED STATES OF ...| 1972-03-08|
|     F-4|UNITED STATES OF ...| 1971-12-27|
|     A-7|UNITED STATES OF ...| 1972-05-24|
|   EC-47|UNITED STATES OF ...| 1972-09-12|
|   CH-53|UNITED STATES OF ...| 1974-06-13|
|   CH-53|UNITED STATES OF ...| 1974-12-19|
|     O-1|     VIETNAM (SOUTH)| 1973-10-24|
|    UH-1|     VIETNAM (SOUTH)| 1974-03-19|
|     C-7|UNITED STATES OF ...| 1970-05-08|
|     A-6|UNITED STATES OF ...| 1971-05-12|
|   EB-66|UNITED STATES OF ...| 1971-12-03|
|    T-28|                LAOS| 1971-12-19|
|     A-6|UNITED STATES OF ...| 1972-08-18|
|     A-7|UNITED STATES OF ...| 

In [21]:
b1=Bombing_operation.select("AirCraft","ContryFlyingMission","MissionDate")

In [22]:
b1.show()

+--------+--------------------+-----------+
|AirCraft| ContryFlyingMission|MissionDate|
+--------+--------------------+-----------+
|   EC-47|UNITED STATES OF ...| 1971-06-05|
|   EC-47|UNITED STATES OF ...| 1972-12-26|
|    RF-4|UNITED STATES OF ...| 1973-07-28|
|     A-1|UNITED STATES OF ...| 1970-02-02|
|    A-37|     VIETNAM (SOUTH)| 1970-10-08|
|     F-4|UNITED STATES OF ...| 1970-11-25|
|     A-4|UNITED STATES OF ...| 1972-03-08|
|     F-4|UNITED STATES OF ...| 1971-12-27|
|     A-7|UNITED STATES OF ...| 1972-05-24|
|   EC-47|UNITED STATES OF ...| 1972-09-12|
|   CH-53|UNITED STATES OF ...| 1974-06-13|
|   CH-53|UNITED STATES OF ...| 1974-12-19|
|     O-1|     VIETNAM (SOUTH)| 1973-10-24|
|    UH-1|     VIETNAM (SOUTH)| 1974-03-19|
|     C-7|UNITED STATES OF ...| 1970-05-08|
|     A-6|UNITED STATES OF ...| 1971-05-12|
|   EB-66|UNITED STATES OF ...| 1971-12-03|
|    T-28|                LAOS| 1971-12-19|
|     A-6|UNITED STATES OF ...| 1972-08-18|
|     A-7|UNITED STATES OF ...| 

Executing Queries Where

In [23]:
b1.where("ContryFlyingMission=='VIETNAM (SOUTH)'").show()

+--------+-------------------+-----------+
|AirCraft|ContryFlyingMission|MissionDate|
+--------+-------------------+-----------+
|    A-37|    VIETNAM (SOUTH)| 1970-10-08|
|     O-1|    VIETNAM (SOUTH)| 1973-10-24|
|    UH-1|    VIETNAM (SOUTH)| 1974-03-19|
|   CH-47|    VIETNAM (SOUTH)| 1971-03-29|
|   AC-47|    VIETNAM (SOUTH)| 1971-09-20|
|    UH-1|    VIETNAM (SOUTH)| 1972-08-22|
|    UH-1|    VIETNAM (SOUTH)| 1973-05-13|
|   CH-47|    VIETNAM (SOUTH)| 1975-01-04|
|    A-37|    VIETNAM (SOUTH)| 1971-02-08|
|   C-119|    VIETNAM (SOUTH)| 1970-09-17|
|    A-37|    VIETNAM (SOUTH)| 1971-04-02|
|     A-1|    VIETNAM (SOUTH)| 1972-04-29|
|     A-1|    VIETNAM (SOUTH)| 1972-08-02|
|    UH-1|    VIETNAM (SOUTH)| 1973-10-24|
|    UH-1|    VIETNAM (SOUTH)| 1974-12-16|
|     A-1|    VIETNAM (SOUTH)| 1970-04-27|
|    UH-1|    VIETNAM (SOUTH)| 1975-02-10|
|    A-37|    VIETNAM (SOUTH)| 1971-03-03|
|     O-1|    VIETNAM (SOUTH)| 1970-08-29|
|     A-1|    VIETNAM (SOUTH)| 1971-07-29|
+--------+-

In [24]:
b1.where("ContryFlyingMission=='VIETNAM (SOUTH)' OR ContryFlyingMission=='LAOS'").show()

+--------+-------------------+-----------+
|AirCraft|ContryFlyingMission|MissionDate|
+--------+-------------------+-----------+
|    A-37|    VIETNAM (SOUTH)| 1970-10-08|
|     O-1|    VIETNAM (SOUTH)| 1973-10-24|
|    UH-1|    VIETNAM (SOUTH)| 1974-03-19|
|    T-28|               LAOS| 1971-12-19|
|   CH-47|    VIETNAM (SOUTH)| 1971-03-29|
|   AC-47|    VIETNAM (SOUTH)| 1971-09-20|
|    UH-1|    VIETNAM (SOUTH)| 1972-08-22|
|    UH-1|    VIETNAM (SOUTH)| 1973-05-13|
|   CH-47|    VIETNAM (SOUTH)| 1975-01-04|
|    A-37|    VIETNAM (SOUTH)| 1971-02-08|
|    T-28|               LAOS| 1972-05-07|
|   C-119|    VIETNAM (SOUTH)| 1970-09-17|
|    A-37|    VIETNAM (SOUTH)| 1971-04-02|
|     A-1|    VIETNAM (SOUTH)| 1972-04-29|
|     A-1|    VIETNAM (SOUTH)| 1972-08-02|
|    UH-1|    VIETNAM (SOUTH)| 1973-10-24|
|    UH-1|    VIETNAM (SOUTH)| 1974-12-16|
|     A-1|    VIETNAM (SOUTH)| 1970-04-27|
|    UH-1|    VIETNAM (SOUTH)| 1975-02-10|
|    A-37|    VIETNAM (SOUTH)| 1971-03-03|
+--------+-

Executing queries in form of sql

In [25]:
Bombing_operation.createOrReplaceTempView("Bombing_operations")

query=  """
        SELECT AirCraft,ContryFlyingMission,MissionDate
        FROM Bombing_operations
        WHERE ContryFlyingMission=='VIETNAM (SOUTH)' OR ContryFlyingMission=='LAOS'
        """
result=spark.sql(query)
result.show()

+--------+-------------------+-----------+
|AirCraft|ContryFlyingMission|MissionDate|
+--------+-------------------+-----------+
|    A-37|    VIETNAM (SOUTH)| 1970-10-08|
|     O-1|    VIETNAM (SOUTH)| 1973-10-24|
|    UH-1|    VIETNAM (SOUTH)| 1974-03-19|
|    T-28|               LAOS| 1971-12-19|
|   CH-47|    VIETNAM (SOUTH)| 1971-03-29|
|   AC-47|    VIETNAM (SOUTH)| 1971-09-20|
|    UH-1|    VIETNAM (SOUTH)| 1972-08-22|
|    UH-1|    VIETNAM (SOUTH)| 1973-05-13|
|   CH-47|    VIETNAM (SOUTH)| 1975-01-04|
|    A-37|    VIETNAM (SOUTH)| 1971-02-08|
|    T-28|               LAOS| 1972-05-07|
|   C-119|    VIETNAM (SOUTH)| 1970-09-17|
|    A-37|    VIETNAM (SOUTH)| 1971-04-02|
|     A-1|    VIETNAM (SOUTH)| 1972-04-29|
|     A-1|    VIETNAM (SOUTH)| 1972-08-02|
|    UH-1|    VIETNAM (SOUTH)| 1973-10-24|
|    UH-1|    VIETNAM (SOUTH)| 1974-12-16|
|     A-1|    VIETNAM (SOUTH)| 1970-04-27|
|    UH-1|    VIETNAM (SOUTH)| 1975-02-10|
|    A-37|    VIETNAM (SOUTH)| 1971-03-03|
+--------+-

In [26]:
query1="""SELECT COUNT(*) AS NUM_OF_ROWS FROM Bombing_operations"""
res=spark.sql(query1)
res.show()

+-----------+
|NUM_OF_ROWS|
+-----------+
|    4400775|
+-----------+



# **Lab Assignment**

## **Problem-1**

Find the Air craft, country flying mission and target country for those missions in which time on target was less than 1000.

In [27]:
query=  """
        SELECT AirCraft,ContryFlyingMission,TargetCountry
        FROM Bombing_operations
        WHERE TimeOnTarget<1000
        """
result=spark.sql(query)
result.show()

+--------+--------------------+-------------+
|AirCraft| ContryFlyingMission|TargetCountry|
+--------+--------------------+-------------+
|   EC-47|UNITED STATES OF ...|SOUTH VIETNAM|
|    RF-4|UNITED STATES OF ...|         LAOS|
|     F-4|UNITED STATES OF ...|         LAOS|
|     F-4|UNITED STATES OF ...|         LAOS|
|     A-7|UNITED STATES OF ...|NORTH VIETNAM|
|   EC-47|UNITED STATES OF ...|SOUTH VIETNAM|
|   CH-53|UNITED STATES OF ...|     THAILAND|
|     O-1|     VIETNAM (SOUTH)|SOUTH VIETNAM|
|    UH-1|     VIETNAM (SOUTH)|SOUTH VIETNAM|
|     C-7|UNITED STATES OF ...|SOUTH VIETNAM|
|    T-28|                LAOS|         LAOS|
|     A-6|UNITED STATES OF ...|NORTH VIETNAM|
|     A-7|UNITED STATES OF ...|NORTH VIETNAM|
|   C-123|UNITED STATES OF ...|SOUTH VIETNAM|
|   CH-47|     VIETNAM (SOUTH)|SOUTH VIETNAM|
|     F-4|UNITED STATES OF ...|SOUTH VIETNAM|
|   AU-24|       KOREA (SOUTH)|     CAMBODIA|
|  AC-130|UNITED STATES OF ...|     CAMBODIA|
|    A-37|UNITED STATES OF ...|   

## **Problem-2**

Find the target country and the number of times they have been attacked. Show the result in descending order.

In [28]:
query=  """
        SELECT TargetCountry, count(*) AS num_of_times
        FROM Bombing_operations
        group by TargetCountry
        order by num_of_times desc;
        """
result=spark.sql(query)
result.show()

+--------------+------------+
| TargetCountry|num_of_times|
+--------------+------------+
| SOUTH VIETNAM|     2172403|
|          LAOS|     1254695|
| NORTH VIETNAM|      694269|
|      CAMBODIA|      218788|
|      THAILAND|       60486|
|WESTPAC WATERS|          74|
|       UNKNOWN|          59|
|   PHILLIPINES|           1|
+--------------+------------+



## **Problem-3**

Find the target country which had been attacked most.

In [29]:
query=  """
        select TargetCountry
        From(SELECT  TargetCountry, count(*) AS num_of_times
        FROM Bombing_operations
        group by TargetCountry
        order by num_of_times desc) LIMIT 1;
        """
result=spark.sql(query)
result.show()

+-------------+
|TargetCountry|
+-------------+
|SOUTH VIETNAM|
+-------------+



## **Problem-4**

Find the country flying mission which attacked most.

In [30]:
query=  """
        select ContryFlyingMission
        From(SELECT  ContryFlyingMission, count(*) AS num_of_times
        FROM Bombing_operations
        group by ContryFlyingMission
        order by num_of_times desc) LIMIT 1;
        """
result=spark.sql(query)
result.show()

+--------------------+
| ContryFlyingMission|
+--------------------+
|UNITED STATES OF ...|
+--------------------+



## **Problem-5**

 Find the take off location and how many times they had used.

In [31]:
query = """
        SELECT TakeoffLocation, count(*) AS numOfTimes
        FROM Bombing_operations
        group by TakeoffLocation
        order by numOfTimes desc
        """
result = spark.sql(query)
result.show()

+---------------+----------+
|TakeoffLocation|numOfTimes|
+---------------+----------+
|       BIEN HOA|    492324|
|         DANANG|    489942|
|        UBON AB|    348822|
|   TAN SON NHUT|    311872|
|       UDORN AB|    306556|
|        CHU LAI|    242096|
|      PHAN RANG|    240238|
|    TONKIN GULF|    194191|
|  NAKHON PHANOM|    194069|
|          KORAT|    191081|
|      BINH THUY|    144003|
|        TUY HOA|    122547|
|         PLEIKU|    110322|
|   CAM RANH BAY|     96373|
|        U TAPAO|     85041|
|           VTBI|     75873|
|         TAKHLI|     65435|
|      NHA TRANG|     56818|
|           VVPU|     53814|
|        PHU CAT|     50353|
+---------------+----------+
only showing top 20 rows



## **Problem-6**

Find the number of fighter jets from Aircraft_Glossary.

In [32]:
Aircraft_glossary.createOrReplaceTempView("Aircraft_Glossary")

In [33]:
query = """
        SELECT  count(AirCraftType) as Fighter_Jet
        FROM Aircraft_Glossary
        where AirCraftType=='Fighter Jet' or AirCraftType=='Fighter jet ' ;
        """
result = spark.sql(query)
result.show()

+-----------+
|Fighter_Jet|
+-----------+
|         11|
+-----------+



## **Problem-7**

Find the number of different types air craft for each air craft type.

In [34]:
query=  """
        SELECT AirCraftType, count(*) AS num_of_times
        FROM Aircraft_glossary
        group by AirCraftType
        """
result=spark.sql(query)
result.show()

+--------------------+------------+
|        AirCraftType|num_of_times|
+--------------------+------------+
|Carrier-based Fig...|           1|
|  Utility Prop plane|           1|
|Early Warning air...|           2|
|Military Transpor...|          11|
|Heavy strategic b...|           1|
|Airliner/transpor...|           3|
|        Light Bomber|           3|
|  Transport Aircraft|           1|
|    Cargo Prop plane|           1|
| Airliner/ Transport|           1|
|Airborne early wa...|           1|
|  Jet Fighter Bomber|           1|
|Weather reconnais...|           1|
|Light Observation...|           1|
|Fixed wing ground...|           1|
|        Fighter jet |           1|
|Photographic reco...|           1|
| Light Utility Plane|           3|
|Close air support...|           1|
|     Tactical Bomber|           1|
+--------------------+------------+
only showing top 20 rows



## **Problem-8**

Find the air craft name which had used most in attacks.

In [35]:
query=  """
        SELECT AirCraft
           from(SELECT b.AirCraft , count(*) as most_use
        FROM Aircraft_glossary a,Bombing_operations b
        where a.AirCraft=b.AirCraft
        group by b.AirCraft
        order by most_use desc) LIMIT 1

        """
result=spark.sql(query)
result.show()

+--------+
|AirCraft|
+--------+
|     F-4|
+--------+

